<div align = "center">

# **Exploración de datos (Base siniestros)**

</div>

## Librerias

In [17]:
library(readxl)
library(dplyr)
library(tidyr)

## Data

In [18]:
Siniestros <- read_excel("../data/input/Siniestros_Hist.xlsx")

In [19]:
# Ver nombres actuales del dataframe Siniestros
names(Siniestros)

[1] "FECHASIN"         "VLRPRIMAPAG"      "VLRPRISUSCR"      "VLRASEGU"        
 [5] "VLRASEGU_ORIG"    "VLRPAGADO."       "VLRSININCUR"      "VLRDEDUCIBLE"    
 [9] "VLRRECOBRO"       "VLRRSVACONSTIAMP" "VLRRSVAPAGAMP"    "FECPAGOAMP"      
[13] "ESTADO_FINAL"     "COBERTURA_FINAL"

In [20]:
# Método directo para renombrar columnas
names(Siniestros) <- c(
  "Fecha_siniestro",
  "Prima_efectivamente_pagada_hasta_fecha_siniestro",
  "Prima_suscrita_incluida_cobertura_responsabilidad_civil",
  "Asegurado_otras_coberturas_distintas_responsabilidad_civil",
  "Asegurado_original",
  "Efectivamente_pagado_por_siniestro",
  "Siniestro_incurrido",
  "Deducible",
  "Recobro_obtenido",
  "Reserva_constituida_para_amparo",
  "Reserva_pagada_para_amparo",
  "Fecha_pago_amparo",
  "Estado_final_siniestro",
  "Cobertura_final_aplicada"
)

In [21]:
head(Siniestros)

Fecha_siniestro,Prima_efectivamente_pagada_hasta_fecha_siniestro,Prima_suscrita_incluida_cobertura_responsabilidad_civil,Asegurado_otras_coberturas_distintas_responsabilidad_civil,Asegurado_original,Efectivamente_pagado_por_siniestro,Siniestro_incurrido,Deducible,Recobro_obtenido,Reserva_constituida_para_amparo,Reserva_pagada_para_amparo,Fecha_pago_amparo,Estado_final_siniestro,Cobertura_final_aplicada
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
2017-01-01,300000,300000,4.350e+06,4.350e+06,0,3262500,870000,0,4.350e+06,3480000,20171220,Pagado,PTH
2017-01-01,5543164,16639492,8.400e+07,8.400e+07,4601532,4601532,737717,0,8.400e+07,0,20170215,Pagado,PPD
2017-01-01,1072019,1072019,5.255e+07,5.255e+07,3375620,3375620,737717,375909,5.255e+07,0,20170131,Pagado,PPD
2017-01-01,958821,958821,3.449e+07,3.449e+07,3600357,3600357,737717,0,3.449e+07,0,20170131,Pagado,PPD
2017-01-01,714176,3398721,2.000e+09,2.000e+09,0,2175223,0,1241379,2.000e+09,2175223,NA,Avisado,PPD
2017-01-01,1681845,5193649,1.500e+09,1.500e+09,1569899,1569899,737717,0,1.500e+09,0,20170213,Pagado,PPD


In [22]:
unique(Siniestros$Cobertura_final_aplicada)

[1] "PTH"       "PPD"       "RC BIENES" "PPH"       "RC PERS"

In [23]:
# Cambiar los niveles de la columna Cobertura_final_aplicada
# No se tiene en cuenta la cobertura de responsabilidad civil por separado, por lo que se agrupan en una sola cobertura

Siniestros$Cobertura_final_aplicada <- factor(Siniestros$Cobertura_final_aplicada, 
                                           levels = c("PTH", "PPD", "RC BIENES", "PPH", "RC PERS"),
                                           labels = c("Perdida_total_hurto", 
                                                     "Perdida_parcial_danos",
                                                     "Responsabilidad_civil",
                                                     "Perdida_parcial_hurto",
                                                     "Responsabilidad_civil"))

In [24]:
# Verigficar los cambios
unique(Siniestros$Cobertura_final_aplicada)

[1] Perdida_total_hurto   Perdida_parcial_danos Responsabilidad_civil
[4] Perdida_parcial_hurto
4 Levels: Perdida_total_hurto Perdida_parcial_danos ... Perdida_parcial_hurto

In [25]:
# Primero corregir el problema de variables - usar Siniestros consistentemente
# Crear columnas de fecha
Siniestros$Año <- format(Siniestros$Fecha_siniestro, "%Y")
Siniestros$Mes <- format(Siniestros$Fecha_siniestro, "%m")
Siniestros$Día <- format(Siniestros$Fecha_siniestro, "%d")
Siniestros$Periodo <- as.numeric(format(Siniestros$Fecha_siniestro, "%Y")) * 100 + as.numeric(format(Siniestros$Fecha_siniestro, "%m"))

# Verificar los tipos de cobertura únicos
head(Siniestros)

Fecha_siniestro,Prima_efectivamente_pagada_hasta_fecha_siniestro,Prima_suscrita_incluida_cobertura_responsabilidad_civil,Asegurado_otras_coberturas_distintas_responsabilidad_civil,Asegurado_original,Efectivamente_pagado_por_siniestro,Siniestro_incurrido,Deducible,Recobro_obtenido,Reserva_constituida_para_amparo,Reserva_pagada_para_amparo,Fecha_pago_amparo,Estado_final_siniestro,Cobertura_final_aplicada,Año,Mes,Día,Periodo
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<chr>,<chr>,<chr>,<dbl>
2017-01-01,300000,300000,4.350e+06,4.350e+06,0,3262500,870000,0,4.350e+06,3480000,20171220,Pagado,Perdida_total_hurto,2017,01,01,201701
2017-01-01,5543164,16639492,8.400e+07,8.400e+07,4601532,4601532,737717,0,8.400e+07,0,20170215,Pagado,Perdida_parcial_danos,2017,01,01,201701
2017-01-01,1072019,1072019,5.255e+07,5.255e+07,3375620,3375620,737717,375909,5.255e+07,0,20170131,Pagado,Perdida_parcial_danos,2017,01,01,201701
2017-01-01,958821,958821,3.449e+07,3.449e+07,3600357,3600357,737717,0,3.449e+07,0,20170131,Pagado,Perdida_parcial_danos,2017,01,01,201701
2017-01-01,714176,3398721,2.000e+09,2.000e+09,0,2175223,0,1241379,2.000e+09,2175223,NA,Avisado,Perdida_parcial_danos,2017,01,01,201701
2017-01-01,1681845,5193649,1.500e+09,1.500e+09,1569899,1569899,737717,0,1.500e+09,0,20170213,Pagado,Perdida_parcial_danos,2017,01,01,201701


In [26]:
## Limpieza de los datos con los que se trabajan inicialmente

### 1. en las celdas anteriores, se renomnraron tanto las columnas, como el contenido de la columna Cobertura_final_aplicada
### 2. se crearon las variables Año, Mes, Día y Periodo, que sirven para identificar los siniestros por año y mes, y hacer el cruce con los datos de IPC_Update por Periodo de acción.
### 3. ahora, vamos a filtrar las columnas que noc coinciden según los dataframes de polizas vendidad y polizas siniestradas, que son solamente 2018

In [27]:
# Filtrar los datos para incluir solo los siniestros de 2018
Siniestros <- Siniestros %>%
    filter(format(Fecha_siniestro, "%Y") == "2018")

# Verificar los resultados
print(min(Siniestros$Fecha_siniestro))
print(max(Siniestros$Fecha_siniestro))

[1] "2018-01-01 UTC"
[1] "2018-12-31 UTC"


In [28]:
IPC_Update <- read.csv2("../data/input/IPC_Update.csv", stringsAsFactors = FALSE)

In [29]:
print(IPC_Update)

   Anio        Mes Mes_Num    Key Variacion Variacion_unidad IPCS_Update
1  2018      Enero       1 201801      0.63           1.0063    1.031721
2  2018    Febrero       2 201802      0.71           1.0071    1.025262
3  2018      Marzo       3 201803      0.24           1.0024    1.018034
4  2018      Abril       4 201804      0.46           1.0046    1.015597
5  2018       Mayo       5 201805      0.25           1.0025    1.010946
6  2018      Junio       6 201806      0.15           1.0015    1.008425
7  2018      Julio       7 201807     -0.13           0.9987    1.006915
8  2018     Agosto       8 201808      0.12           1.0012    1.008226
9  2018 Septiembre       9 201809      0.16           1.0016    1.007017
10 2018    Octubre      10 201810      0.12           1.0012    1.005409
11 2018  Noviembre      11 201811      0.12           1.0012    1.004204
12 2018  Diciembre      12 201812      0.30           1.0030    1.003000
13 2019      Enero       1 201901      0.60        

In [30]:
Siniestros <- Siniestros %>%
select(Fecha_siniestro, Año, Mes, Día, Periodo, Siniestro_incurrido, Cobertura_final_aplicada)
head(Siniestros)

Fecha_siniestro,Año,Mes,Día,Periodo,Siniestro_incurrido,Cobertura_final_aplicada
<dttm>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<fct>
2018-01-01,2018,01,01,201801,1735248,Perdida_parcial_danos
2018-01-01,2018,01,01,201801,1601730,Perdida_parcial_danos
2018-01-01,2018,01,01,201801,747874,Perdida_parcial_danos
2018-01-01,2018,01,01,201801,5555977,Perdida_parcial_danos
2018-01-01,2018,01,01,201801,3400985,Perdida_parcial_danos
2018-01-01,2018,01,01,201801,3727847,Perdida_parcial_danos


In [31]:
Siniestros <- Siniestros %>%
left_join(IPC_Update %>% select(Key, IPCS_Update), by = c("Periodo" = "Key")) %>%
mutate(Siniestro_incurrido_update = Siniestro_incurrido * IPCS_Update)
head(Siniestros)

Fecha_siniestro,Año,Mes,Día,Periodo,Siniestro_incurrido,Cobertura_final_aplicada,IPCS_Update,Siniestro_incurrido_update
<dttm>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
2018-01-01,2018,01,01,201801,1735248,Perdida_parcial_danos,1.031721,1790292.6
2018-01-01,2018,01,01,201801,1601730,Perdida_parcial_danos,1.031721,1652539.2
2018-01-01,2018,01,01,201801,747874,Perdida_parcial_danos,1.031721,771597.6
2018-01-01,2018,01,01,201801,5555977,Perdida_parcial_danos,1.031721,5732220.6
2018-01-01,2018,01,01,201801,3400985,Perdida_parcial_danos,1.031721,3508869.2
2018-01-01,2018,01,01,201801,3727847,Perdida_parcial_danos,1.031721,3846099.7


In [32]:
#Perdida parcial por daños

Siniestros_ppd <- Siniestros %>%
filter(Cobertura_final_aplicada == "Perdida_parcial_danos") %>%
group_by(Año, Mes, Día, Periodo) %>%
summarise(
    Cantidad_siniestros = n(),
    Valor_siniestro_incurrido_update = sum(Siniestro_incurrido_update, na.rm = TRUE),
    .groups = 'drop'
)

head(Siniestros_ppd)

Año,Mes,Día,Periodo,Cantidad_siniestros,Valor_siniestro_incurrido_update
<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>
2018,01,01,201801,59,231334801
2018,01,02,201801,64,237311818
2018,01,03,201801,52,211603698
2018,01,04,201801,62,289397034
2018,01,05,201801,39,185970337
2018,01,06,201801,56,221639483


In [33]:
# Pérdida total por hurto

Siniestros_pth <- Siniestros %>%
filter(Cobertura_final_aplicada == "Perdida_total_hurto") %>%
group_by(Año, Mes, Día, Periodo) %>%
summarise(
    Cantidad_siniestros = n(),
    Valor_siniestro_incurrido_update = sum(Siniestro_incurrido_update, na.rm = TRUE),
    .groups = 'drop'
)

head(Siniestros_pth)

Año,Mes,Día,Periodo,Cantidad_siniestros,Valor_siniestro_incurrido_update
<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>
2018,01,01,201801,1,0
2018,01,03,201801,1,3300683
2018,01,04,201801,1,0
2018,01,05,201801,2,7580062
2018,01,06,201801,1,5076069
2018,01,07,201801,2,6116045


In [34]:
# Pérdida parcial por hurto
Siniestros_pph <- Siniestros %>%
filter(Cobertura_final_aplicada == "Perdida_parcial_hurto") %>%
group_by(Año, Mes, Día, Periodo) %>%
summarise(
    Cantidad_siniestros = n(),
    Valor_siniestro_incurrido_update = sum(Siniestro_incurrido_update, na.rm = TRUE),
    .groups = 'drop'
)

head(Siniestros_pph)

Año,Mes,Día,Periodo,Cantidad_siniestros,Valor_siniestro_incurrido_update
<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>
2018,01,01,201801,1,1462347.5
2018,01,02,201801,2,7327703.5
2018,01,04,201801,3,477045.3
2018,01,05,201801,2,2176136.8
2018,01,06,201801,1,4661773.5
2018,01,08,201801,2,5522289.0


In [35]:
# Responsabilidad civil
Siniestros_rcb <- Siniestros %>%
  filter(Cobertura_final_aplicada == "Responsabilidad_civil") %>%
  group_by(Año, Mes, Día, Periodo) %>%
  summarise(
    Cantidad_siniestros = n(),
    Valor_siniestro_incurrido_update = sum(Siniestro_incurrido_update, na.rm = TRUE),
    .groups = 'drop'
  )

head(Siniestros_rcb)

Año,Mes,Día,Periodo,Cantidad_siniestros,Valor_siniestro_incurrido_update
<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>
2018,01,01,201801,1,6699343
2018,01,02,201801,11,26066327
2018,01,03,201801,16,32570303
2018,01,04,201801,7,14809207
2018,01,05,201801,16,92102331
2018,01,06,201801,8,11120532


In [37]:
# Guardar las tablas pivoteadas como archivos CSV
write.csv(Siniestros_pth, "../data/processed/Siniestros_pth.csv", row.names = FALSE)
write.csv(Siniestros_ppd, "../data/processed/Siniestros_ppd.csv", row.names = FALSE)
write.csv(Siniestros_rcb, "../data/processed/Siniestros_rcb.csv", row.names = FALSE)
write.csv(Siniestros_pph, "../data/processed/Siniestros_pph.csv", row.names = FALSE)